# Web Crawling 


Se realiza el crawling con beautiful soup para las pagínas de cursos GCFGlobal:
* https://edu.gcfglobal.org/en/topics/
* https://edu.gcfglobal.org/pt/topics/
* https://edu.gcfglobal.org/es/topics/


## Crawler

In [1]:
import requests
import time
import pandas as pd
from bs4 import BeautifulSoup

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!ls drive

MyDrive  Othercomputers


In [46]:
root = 'https://edu.gcfglobal.org'
urls = ['https://edu.gcfglobal.org/es/topics/',
          'https://edu.gcfglobal.org/en/topics/',
           'https://edu.gcfglobal.org/pt/topics/']
all_link = []
temas =[]
for i in urls:
  html_text = requests.get(i).text
  soup = BeautifulSoup(html_text,'html.parser') 
  anchor = soup.find_all('a', class_= 'all-topics-tutorial-link')
  colecciones = soup.find_all('li', class_='supercollection all-topics')
  for coleccion in colecciones:
    tema= coleccion.find('a', class_='all-topics-subject-link').text # encuentra el primero
    cursos = coleccion.find_all('a', class_='all-topics-tutorial-link')
    for link in cursos:
      if('https' not in link.get('href')):
        linktest = root + link.get('href')
      else:
        linktest = link.get('href')
      temas.append(tema)
      all_link.append(linktest)
      print(linktest, tema)
          #print("The size of the pages is: ", len(all_link))

https://edu.gcfglobal.org/es/informatica-basica/ Informática básica
https://edu.gcfglobal.org/es/informatica-avanzada/ Informática básica
https://edu.gcfglobal.org/es/como-imprimir/ Informática básica
https://edu.gcfglobal.org/es/virus-informaticos-y-antivirus/ Informática básica
https://edu.gcfglobal.org/es/curso-basico-de-html/ Informática básica
https://edu.gcfglobal.org/es/procesadores-de-texto/ Ofimática
https://edu.gcfglobal.org/es/presentaciones-basicas/ Ofimática
https://edu.gcfglobal.org/es/programas-de-hojas-de-calculo/ Ofimática
https://edu.gcfglobal.org/es/google-drive/ Ofimática
https://edu.gcfglobal.org/es/google-slides/ Ofimática
https://edu.gcfglobal.org/es/google-docs/ Ofimática
https://edu.gcfglobal.org/es/google-meet/ Ofimática
https://edu.gcfglobal.org/es/como-usar-internet/ Internet
https://edu.gcfglobal.org/es/seguridad-en-internet/ Internet
https://edu.gcfglobal.org/es/crear-un-blog-en-internet/ Internet
https://edu.gcfglobal.org/es/creacion-de-sitios-web/ Intern

In [50]:
len(all_link), len(temas)

(487, 487)

In [52]:
all_link[1], temas[1]

('https://edu.gcfglobal.org/es/informatica-avanzada/', 'Informática básica')

In [53]:
def get_course_link(link):
  r = requests.get(link)
  soup = BeautifulSoup(r.text,'html.parser')
  course_header1 = soup.find('div', class_= 'sub-header')
  info={}
  if course_header1:
    info['titulo'] = course_header1.find('h1').text.replace('\n', '')
    info['descripcion'] = course_header1.find('p').text.replace('\n', '')
    anchor_link = course_header1.find_all('a')
    for a in anchor_link:
      if('https' not in a.get('href')):
        info['inicio_curso'] = 'https://idiomas.gcfglobal.org' + a.get('href')
      # else:
      #   print(f"Header different!: {link}" )
  else: 
    print(f"Header different!: {link}" )
  return info

In [54]:
get_course_link('https://edu.gcfglobal.org/en/basicspanishskills/')

{'titulo': 'Basic Spanish Skills',
 'descripcion': 'This free tutorial covers some of the most common Spanish greetings and phrases.'}

In [55]:
def get_course_info(link):
    info = {}
    r = requests.get(link)
    if r.status_code == 200:
        soup = BeautifulSoup(r.text,'html.parser')
        course_header = soup.find('div', class_= 'header-desc')
        if course_header:
            info['titulo'] = course_header.find('h1').text
            desc = course_header.find('div', class_= 'description')
            parrafos = desc.find_all('p')
            anchor_link = course_header.find_all('a')
            desc_text = ""
            for parrafo in parrafos:
                desc_text += parrafo.text + "\n"
            info['descripcion'] = desc_text[:-1]
            for a in anchor_link:
              if('https' not in a.get('href')):
                info['inicio_curso'] = root + a.get('href')
              else: 
                info['videos_youtube']  = a.get('href')
        else:
              info = get_course_link(link)
              print(f"Header different!: {link}")             
    else:
        print(f"Connection failed to {link}")
    return info

In [56]:
get_course_info('https://edu.gcfglobal.org/en/basicspanishskills/')


{'titulo': 'Basic Spanish Skills',
 'descripcion': 'This free tutorial covers some of the most common Spanish greetings and phrases.'}

In [57]:
get_course_link('https://edu.gcfglobal.org/en/basicspanishskills/')

{'titulo': 'Basic Spanish Skills',
 'descripcion': 'This free tutorial covers some of the most common Spanish greetings and phrases.'}

In [58]:
def get_course_lesson(link):
  info={}
  r = requests.get(link)
  if r.status_code == 200:
    soup = BeautifulSoup(r.text,'html.parser')
    course_lesson = soup.find('div', class_= 'collection-content')
    course_lesson1 = soup.find('div', class_= 'tutorial-info')
    if course_lesson:
      lessons = course_lesson.find_all('h3')
      texto = ""
      for l in lessons:
        texto += l.text + ","
        info['lecciones']= texto[:-1]
    elif course_lesson1:
      print(f"lesson not found {link}")
      lessons = course_lesson1.find_all('p')
      texto1 = ""
      for l1 in lessons:
        texto1 += l1.text + ","
        info['lecciones']= texto1[:-1].replace('\n', '')
  else:
    print(f"Connection failed to {link}")
  return info

In [59]:
get_course_lesson('https://edu.gcfglobal.org/en/basicspanishskills/')

{'lecciones': 'Basic Conversation Skills,Using Spanish in Real Life'}

In [60]:
all_courses_info = []

for link in all_link:
    course_info = get_course_info(link)
    all_courses_info.append(course_info)
    print(course_info)
    time.sleep(0.5)


{'titulo': 'Informática Básica', 'descripcion': 'Aprende cómo empezar a usar un computador desde el principio, cuáles son sus partes, cómo conectarlo e instalar programas.', 'inicio_curso': 'https://edu.gcfglobal.org/es/informatica-basica/que-es-un-computador/1/', 'videos_youtube': 'https://www.youtube.com/playlist?list=PLQVIjL9XTrQ0grV_RVr8Fw2LxYNHc-jVt'}
{'titulo': 'Conceptos básicos de programación', 'descripcion': 'Aprende los principios y conceptos de la programación. Conoce todo sobre los lenguajes de programación y comprende cómo se desarrollan los sistemas informáticos.', 'inicio_curso': 'https://edu.gcfglobal.org/es/conceptos-basicos-de-programacion/que-es-programar/1/', 'videos_youtube': 'https://www.youtube.com/watch?v=3F-kuNGlNco&list=PLQVIjL9XTrQ1ANovnlN8WbHp-YxWdzKYK'}
{'titulo': '¿Cómo imprimir?', 'descripcion': 'Aprende los pasos básicos para imprimir desde tu computador un documento, foto, página web o correo electrónico. Además, te enseñamos la forma más sencilla de i

In [63]:
dataGCF = pd.DataFrame.from_dict(all_courses_info)
dataGCF['URL'] = all_link
dataGCF['tema'] = temas
dataGCF

,titulo,descripcion,inicio_curso,videos_youtube,URL,tema
0,Informática Básica,Aprende cómo empezar a usar un computador desd...,https://edu.gcfglobal.org/es/informatica-basic...,https://www.youtube.com/playlist?list=PLQVIjL9...,https://edu.gcfglobal.org/es/informatica-basica/,Informática básica
1,Conceptos básicos de programación,Aprende los principios y conceptos de la progr...,https://edu.gcfglobal.org/es/conceptos-basicos...,https://www.youtube.com/watch?v=3F-kuNGlNco&li...,https://edu.gcfglobal.org/es/informatica-avanz...,Informática básica
2,¿Cómo imprimir?,Aprende los pasos básicos para imprimir desde ...,https://edu.gcfglobal.org/es/como-imprimir/com...,NaN,https://edu.gcfglobal.org/es/como-imprimir/,Informática básica
3,Virus informáticos y antivirus,En este curso verás qué son son los virus info...,https://edu.gcfglobal.org/es/virus-informatico...,NaN,https://edu.gcfglobal.org/es/virus-informatico...,Informática básica
4,Curso básico de HTML,"Aprende los conceptos básicos del HTML, un len...",https://edu.gcfglobal.org/es/curso-basico-de-h...,NaN,https://edu.gcfglobal.org/es/curso-basico-de-h...,Informática básica
...,...,...,...,...,...,...
482,a2. Básico,Aprenda a expressar todas as suas idéias em in...,https://idiomas.gcfglobal.org/pt/curso/ingles/...,NaN,https://idiomas.gcfglobal.org/pt/curso/ingles/a2/,Inglês
483,b1. Intermediário,"Aprenda a usar corretamente os conectivos, ver...",https://idiomas.gcfglobal.org/pt/curso/ingles/...,NaN,https://idiomas.gcfglobal.org/pt/curso/ingles/b1/,Inglês
484,b2. Intermediário avançado,"Aqui você verá temas como voz passiva e ativa,...",https://idiomas.gcfglobal.org/pt/curso/ingles/...,NaN,https://idiomas.gcfglobal.org/pt/curso/ingles/b2/,Inglês
485,c1. Avançado,"Neste nível avançado do curso, você já poderá ...",https://idiomas.gcfglobal.org/pt/curso/ingles/...,NaN,https://idiomas.gcfglobal.org/pt/curso/ingles/c1/,Inglês


In [62]:
all_courses_lesson = []
for link in all_link:
    course_lesson = get_course_lesson(link)
    all_courses_lesson.append(course_lesson)
    print(course_lesson)
    time.sleep(0.5)

{'lecciones': 'Cómo usar un computador,Explora Windows,Explora Mac OS X,Las partes básicas de un computador,Sistemas operativos,Computadores Portátiles,Otros tipos de dispositivos,Dispositivos móviles,Otros Conceptos'}
{'lecciones': '¿Qué es la programación?,Sistema binario,Códigos numéricos,Tipos de datos y operadores,Algoritmos,Diagramas de flujo,Lenguajes de programación'}
{'lecciones': '¿Qué necesitas para imprimir?,¿Cómo imprimir un archivo?,¿Cómo imprimir desde un teléfono móvil?'}
{'lecciones': 'Los virus informaticos,Los antivirus'}
{'lecciones': '¿Cómo empezar?,Escribe en HTML,Extras'}
{'lecciones': 'Opciones básicas de texto,Función Insertar'}
{'lecciones': 'Presentación básica'}
{'lecciones': 'Composición de una hoja de cálculo,Fórmulas y funciones en libros de cálculo'}
{'lecciones': 'Introducción a Google Drive, Gestión de archivos y carpetas en Google Drive, Google Drive en dispositivos,Test'}
{'lecciones': 'Introducción a Google Slides,Temas y plantillas ,Herramientas bá

In [64]:
dataGCF1 = pd.DataFrame.from_dict(all_courses_lesson)
dataGCF1

,lecciones
0,"Cómo usar un computador,Explora Windows,Explor..."
1,"¿Qué es la programación?,Sistema binario,Códig..."
2,"¿Qué necesitas para imprimir?,¿Cómo imprimir u..."
3,"Los virus informaticos,Los antivirus"
4,"¿Cómo empezar?,Escribe en HTML,Extras"
...,...
482,"Advérbios de lugar em inglêsarrow_forward_ios,..."
483,O que é preposição de tempo em inglês?arrow_fo...
484,"Before and Afterarrow_forward_ios,Aprenda a di..."
485,"English language examsarrow_forward_ios,Você t..."


In [65]:
dataGCFJoin = pd.concat([dataGCF,dataGCF1],axis=1) 
dataGCFJoin.head()

,titulo,descripcion,inicio_curso,videos_youtube,URL,tema,lecciones
0,Informática Básica,Aprende cómo empezar a usar un computador desd...,https://edu.gcfglobal.org/es/informatica-basic...,https://www.youtube.com/playlist?list=PLQVIjL9...,https://edu.gcfglobal.org/es/informatica-basica/,Informática básica,"Cómo usar un computador,Explora Windows,Explor..."
1,Conceptos básicos de programación,Aprende los principios y conceptos de la progr...,https://edu.gcfglobal.org/es/conceptos-basicos...,https://www.youtube.com/watch?v=3F-kuNGlNco&li...,https://edu.gcfglobal.org/es/informatica-avanz...,Informática básica,"¿Qué es la programación?,Sistema binario,Códig..."
2,¿Cómo imprimir?,Aprende los pasos básicos para imprimir desde ...,https://edu.gcfglobal.org/es/como-imprimir/com...,NaN,https://edu.gcfglobal.org/es/como-imprimir/,Informática básica,"¿Qué necesitas para imprimir?,¿Cómo imprimir u..."
3,Virus informáticos y antivirus,En este curso verás qué son son los virus info...,https://edu.gcfglobal.org/es/virus-informatico...,NaN,https://edu.gcfglobal.org/es/virus-informatico...,Informática básica,"Los virus informaticos,Los antivirus"
4,Curso básico de HTML,"Aprende los conceptos básicos del HTML, un len...",https://edu.gcfglobal.org/es/curso-basico-de-h...,NaN,https://edu.gcfglobal.org/es/curso-basico-de-h...,Informática básica,"¿Cómo empezar?,Escribe en HTML,Extras"


In [66]:
# guardar en un csv
#Información GCF
dataGCFJoin.to_csv('drive/MyDrive/Laboratorios SW/Datos pagina/datosGCFraw.csv')  

## Limpieza del Dataset

In [69]:
dataGCFJoin.head(10)

,titulo,descripcion,inicio_curso,videos_youtube,URL,tema,lecciones
0,Informática Básica,Aprende cómo empezar a usar un computador desd...,https://edu.gcfglobal.org/es/informatica-basic...,https://www.youtube.com/playlist?list=PLQVIjL9...,https://edu.gcfglobal.org/es/informatica-basica/,Informática básica,"Cómo usar un computador,Explora Windows,Explor..."
1,Conceptos básicos de programación,Aprende los principios y conceptos de la progr...,https://edu.gcfglobal.org/es/conceptos-basicos...,https://www.youtube.com/watch?v=3F-kuNGlNco&li...,https://edu.gcfglobal.org/es/informatica-avanz...,Informática básica,"¿Qué es la programación?,Sistema binario,Códig..."
2,¿Cómo imprimir?,Aprende los pasos básicos para imprimir desde ...,https://edu.gcfglobal.org/es/como-imprimir/com...,NaN,https://edu.gcfglobal.org/es/como-imprimir/,Informática básica,"¿Qué necesitas para imprimir?,¿Cómo imprimir u..."
3,Virus informáticos y antivirus,En este curso verás qué son son los virus info...,https://edu.gcfglobal.org/es/virus-informatico...,NaN,https://edu.gcfglobal.org/es/virus-informatico...,Informática básica,"Los virus informaticos,Los antivirus"
4,Curso básico de HTML,"Aprende los conceptos básicos del HTML, un len...",https://edu.gcfglobal.org/es/curso-basico-de-h...,NaN,https://edu.gcfglobal.org/es/curso-basico-de-h...,Informática básica,"¿Cómo empezar?,Escribe en HTML,Extras"
5,Procesadores de texto,Aquí aprenderás sobre las herramientas básicas...,https://edu.gcfglobal.org/es/procesadores-de-t...,NaN,https://edu.gcfglobal.org/es/procesadores-de-t...,Ofimática,"Opciones básicas de texto,Función Insertar"
6,Presentaciones básicas,En este curso aprenderás a crear presentacione...,https://edu.gcfglobal.org/es/presentaciones-ba...,https://www.youtube.com/watch?v=YMzDHZmCncQ&li...,https://edu.gcfglobal.org/es/presentaciones-ba...,Ofimática,Presentación básica
7,Programas de hojas de cálculo,Conoce cuáles son las herramientas básicas al ...,https://edu.gcfglobal.org/es/programas-de-hoja...,https://www.youtube.com/watch?v=ZBkkrnD7GBo&li...,https://edu.gcfglobal.org/es/programas-de-hoja...,Ofimática,"Composición de una hoja de cálculo,Fórmulas y ..."
8,Google Drive,Aprende en este curso gratuito de Google Drive...,https://edu.gcfglobal.org/es/google-drive/que-...,NaN,https://edu.gcfglobal.org/es/google-drive/,Ofimática,"Introducción a Google Drive, Gestión de archiv..."
9,Google Slides,En este curso te enseñaremos a usar este progr...,https://edu.gcfglobal.org/es/google-slides/goo...,NaN,https://edu.gcfglobal.org/es/google-slides/,Ofimática,"Introducción a Google Slides,Temas y plantilla..."


In [72]:
df = dataGCFJoin.copy()
df.head(10)

,titulo,descripcion,inicio_curso,videos_youtube,URL,tema,lecciones
0,Informática Básica,Aprende cómo empezar a usar un computador desd...,https://edu.gcfglobal.org/es/informatica-basic...,https://www.youtube.com/playlist?list=PLQVIjL9...,https://edu.gcfglobal.org/es/informatica-basica/,Informática básica,"Cómo usar un computador,Explora Windows,Explor..."
1,Conceptos básicos de programación,Aprende los principios y conceptos de la progr...,https://edu.gcfglobal.org/es/conceptos-basicos...,https://www.youtube.com/watch?v=3F-kuNGlNco&li...,https://edu.gcfglobal.org/es/informatica-avanz...,Informática básica,"¿Qué es la programación?,Sistema binario,Códig..."
2,¿Cómo imprimir?,Aprende los pasos básicos para imprimir desde ...,https://edu.gcfglobal.org/es/como-imprimir/com...,NaN,https://edu.gcfglobal.org/es/como-imprimir/,Informática básica,"¿Qué necesitas para imprimir?,¿Cómo imprimir u..."
3,Virus informáticos y antivirus,En este curso verás qué son son los virus info...,https://edu.gcfglobal.org/es/virus-informatico...,NaN,https://edu.gcfglobal.org/es/virus-informatico...,Informática básica,"Los virus informaticos,Los antivirus"
4,Curso básico de HTML,"Aprende los conceptos básicos del HTML, un len...",https://edu.gcfglobal.org/es/curso-basico-de-h...,NaN,https://edu.gcfglobal.org/es/curso-basico-de-h...,Informática básica,"¿Cómo empezar?,Escribe en HTML,Extras"
5,Procesadores de texto,Aquí aprenderás sobre las herramientas básicas...,https://edu.gcfglobal.org/es/procesadores-de-t...,NaN,https://edu.gcfglobal.org/es/procesadores-de-t...,Ofimática,"Opciones básicas de texto,Función Insertar"
6,Presentaciones básicas,En este curso aprenderás a crear presentacione...,https://edu.gcfglobal.org/es/presentaciones-ba...,https://www.youtube.com/watch?v=YMzDHZmCncQ&li...,https://edu.gcfglobal.org/es/presentaciones-ba...,Ofimática,Presentación básica
7,Programas de hojas de cálculo,Conoce cuáles son las herramientas básicas al ...,https://edu.gcfglobal.org/es/programas-de-hoja...,https://www.youtube.com/watch?v=ZBkkrnD7GBo&li...,https://edu.gcfglobal.org/es/programas-de-hoja...,Ofimática,"Composición de una hoja de cálculo,Fórmulas y ..."
8,Google Drive,Aprende en este curso gratuito de Google Drive...,https://edu.gcfglobal.org/es/google-drive/que-...,NaN,https://edu.gcfglobal.org/es/google-drive/,Ofimática,"Introducción a Google Drive, Gestión de archiv..."
9,Google Slides,En este curso te enseñaremos a usar este progr...,https://edu.gcfglobal.org/es/google-slides/goo...,NaN,https://edu.gcfglobal.org/es/google-slides/,Ofimática,"Introducción a Google Slides,Temas y plantilla..."


In [80]:
df.isnull().sum()
# inicio_curso      302
# descripcion         5
# inicio_curso      302
# videos_youtube    277
# URL                 0
# tema                0
# lecciones          78

titulo              5
descripcion         5
inicio_curso      302
videos_youtube    277
URL                 0
tema                0
lecciones          78
dtype: int64

In [115]:
#Localiza los nulos, en este caso se eliminan las URL caidas y las que contengan nan en las caracteristicas minimas evaluadas
df.loc[(df['titulo'].isnull()) | (df['descripcion'].isnull())]
df.drop(df[df['titulo'].isnull()].index, inplace =True)

,titulo,descripcion,inicio_curso,videos_youtube,URL,tema,lecciones
0,Informática Básica,Aprende cómo empezar a usar un computador desd...,https://edu.gcfglobal.org/es/informatica-basic...,https://www.youtube.com/playlist?list=PLQVIjL9...,https://edu.gcfglobal.org/es/informatica-basica/,Informática básica,"Cómo usar un computador,Explora Windows,Explor..."
1,Conceptos básicos de programación,Aprende los principios y conceptos de la progr...,https://edu.gcfglobal.org/es/conceptos-basicos...,https://www.youtube.com/watch?v=3F-kuNGlNco&li...,https://edu.gcfglobal.org/es/informatica-avanz...,Informática básica,"¿Qué es la programación?,Sistema binario,Códig..."
2,¿Cómo imprimir?,Aprende los pasos básicos para imprimir desde ...,https://edu.gcfglobal.org/es/como-imprimir/com...,NaN,https://edu.gcfglobal.org/es/como-imprimir/,Informática básica,"¿Qué necesitas para imprimir?,¿Cómo imprimir u..."
3,Virus informáticos y antivirus,En este curso verás qué son son los virus info...,https://edu.gcfglobal.org/es/virus-informatico...,NaN,https://edu.gcfglobal.org/es/virus-informatico...,Informática básica,"Los virus informaticos,Los antivirus"
4,Curso básico de HTML,"Aprende los conceptos básicos del HTML, un len...",https://edu.gcfglobal.org/es/curso-basico-de-h...,NaN,https://edu.gcfglobal.org/es/curso-basico-de-h...,Informática básica,"¿Cómo empezar?,Escribe en HTML,Extras"
...,...,...,...,...,...,...,...
481,a1. Iniciante,Neste nível você conhecerá o alfabeto básico p...,https://idiomas.gcfglobal.org/pt/curso/ingles/...,NaN,https://idiomas.gcfglobal.org/pt/curso/ingles/a1/,Inglês,"O alfabeto em inglêsarrow_forward_ios,Aqui voc..."
482,a2. Básico,Aprenda a expressar todas as suas idéias em in...,https://idiomas.gcfglobal.org/pt/curso/ingles/...,NaN,https://idiomas.gcfglobal.org/pt/curso/ingles/a2/,Inglês,"Advérbios de lugar em inglêsarrow_forward_ios,..."
483,b1. Intermediário,"Aprenda a usar corretamente os conectivos, ver...",https://idiomas.gcfglobal.org/pt/curso/ingles/...,NaN,https://idiomas.gcfglobal.org/pt/curso/ingles/b1/,Inglês,O que é preposição de tempo em inglês?arrow_fo...
484,b2. Intermediário avançado,"Aqui você verá temas como voz passiva e ativa,...",https://idiomas.gcfglobal.org/pt/curso/ingles/...,NaN,https://idiomas.gcfglobal.org/pt/curso/ingles/b2/,Inglês,"Before and Afterarrow_forward_ios,Aprenda a di..."


In [134]:
# URL que no posean el link de inicio_curso y videos_youtube  son 168  y se reemplazan por cero
df.loc[(df['inicio_curso'].isnull()) & (df['videos_youtube'].isnull())]
df['inicio_curso'].fillna(0, inplace= True)
df['videos_youtube'].fillna(0, inplace= True)
df

,titulo,descripcion,inicio_curso,videos_youtube,URL,tema,lecciones
0,Informática Básica,Aprende cómo empezar a usar un computador desd...,https://edu.gcfglobal.org/es/informatica-basic...,https://www.youtube.com/playlist?list=PLQVIjL9...,https://edu.gcfglobal.org/es/informatica-basica/,Informática básica,"Cómo usar un computador,Explora Windows,Explor..."
1,Conceptos básicos de programación,Aprende los principios y conceptos de la progr...,https://edu.gcfglobal.org/es/conceptos-basicos...,https://www.youtube.com/watch?v=3F-kuNGlNco&li...,https://edu.gcfglobal.org/es/informatica-avanz...,Informática básica,"¿Qué es la programación?,Sistema binario,Códig..."
2,¿Cómo imprimir?,Aprende los pasos básicos para imprimir desde ...,https://edu.gcfglobal.org/es/como-imprimir/com...,0,https://edu.gcfglobal.org/es/como-imprimir/,Informática básica,"¿Qué necesitas para imprimir?,¿Cómo imprimir u..."
3,Virus informáticos y antivirus,En este curso verás qué son son los virus info...,https://edu.gcfglobal.org/es/virus-informatico...,0,https://edu.gcfglobal.org/es/virus-informatico...,Informática básica,"Los virus informaticos,Los antivirus"
4,Curso básico de HTML,"Aprende los conceptos básicos del HTML, un len...",https://edu.gcfglobal.org/es/curso-basico-de-h...,0,https://edu.gcfglobal.org/es/curso-basico-de-h...,Informática básica,"¿Cómo empezar?,Escribe en HTML,Extras"
...,...,...,...,...,...,...,...
481,a1. Iniciante,Neste nível você conhecerá o alfabeto básico p...,https://idiomas.gcfglobal.org/pt/curso/ingles/...,0,https://idiomas.gcfglobal.org/pt/curso/ingles/a1/,Inglês,"O alfabeto em inglêsarrow_forward_ios,Aqui voc..."
482,a2. Básico,Aprenda a expressar todas as suas idéias em in...,https://idiomas.gcfglobal.org/pt/curso/ingles/...,0,https://idiomas.gcfglobal.org/pt/curso/ingles/a2/,Inglês,"Advérbios de lugar em inglêsarrow_forward_ios,..."
483,b1. Intermediário,"Aprenda a usar corretamente os conectivos, ver...",https://idiomas.gcfglobal.org/pt/curso/ingles/...,0,https://idiomas.gcfglobal.org/pt/curso/ingles/b1/,Inglês,O que é preposição de tempo em inglês?arrow_fo...
484,b2. Intermediário avançado,"Aqui você verá temas como voz passiva e ativa,...",https://idiomas.gcfglobal.org/pt/curso/ingles/...,0,https://idiomas.gcfglobal.org/pt/curso/ingles/b2/,Inglês,"Before and Afterarrow_forward_ios,Aprenda a di..."


In [135]:
df.isnull().sum()

titulo             0
descripcion        0
inicio_curso       0
videos_youtube     0
URL                0
tema               0
lecciones         73
dtype: int64

In [138]:
df.loc[df['lecciones'].isnull()]
df['lecciones'].fillna(0, inplace= True)

In [139]:
df.isnull().sum()

titulo            0
descripcion       0
inicio_curso      0
videos_youtube    0
URL               0
tema              0
lecciones         0
dtype: int64

In [140]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 482 entries, 0 to 485
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   titulo          482 non-null    object
 1   descripcion     482 non-null    object
 2   inicio_curso    482 non-null    object
 3   videos_youtube  482 non-null    object
 4   URL             482 non-null    object
 5   tema            482 non-null    object
 6   lecciones       482 non-null    object
dtypes: object(7)
memory usage: 30.1+ KB


In [141]:
# estadisticas
df.describe()

,titulo,descripcion,inicio_curso,videos_youtube,URL,tema,lecciones
count,482,482,482,482,482,482,482
unique,378,405,186,156,408,90,344
top,Word 2016,"In this free tutorial, learn how to use this M...",0,0,https://edu.gcfglobal.org/en/quickbooks-resour...,Internet,0
freq,4,4,297,272,3,24,73


In [143]:
df.head(50)

,titulo,descripcion,inicio_curso,videos_youtube,URL,tema,lecciones
0,Informática Básica,Aprende cómo empezar a usar un computador desd...,https://edu.gcfglobal.org/es/informatica-basic...,https://www.youtube.com/playlist?list=PLQVIjL9...,https://edu.gcfglobal.org/es/informatica-basica/,Informática básica,"Cómo usar un computador,Explora Windows,Explor..."
1,Conceptos básicos de programación,Aprende los principios y conceptos de la progr...,https://edu.gcfglobal.org/es/conceptos-basicos...,https://www.youtube.com/watch?v=3F-kuNGlNco&li...,https://edu.gcfglobal.org/es/informatica-avanz...,Informática básica,"¿Qué es la programación?,Sistema binario,Códig..."
2,¿Cómo imprimir?,Aprende los pasos básicos para imprimir desde ...,https://edu.gcfglobal.org/es/como-imprimir/com...,0,https://edu.gcfglobal.org/es/como-imprimir/,Informática básica,"¿Qué necesitas para imprimir?,¿Cómo imprimir u..."
3,Virus informáticos y antivirus,En este curso verás qué son son los virus info...,https://edu.gcfglobal.org/es/virus-informatico...,0,https://edu.gcfglobal.org/es/virus-informatico...,Informática básica,"Los virus informaticos,Los antivirus"
4,Curso básico de HTML,"Aprende los conceptos básicos del HTML, un len...",https://edu.gcfglobal.org/es/curso-basico-de-h...,0,https://edu.gcfglobal.org/es/curso-basico-de-h...,Informática básica,"¿Cómo empezar?,Escribe en HTML,Extras"
5,Procesadores de texto,Aquí aprenderás sobre las herramientas básicas...,https://edu.gcfglobal.org/es/procesadores-de-t...,0,https://edu.gcfglobal.org/es/procesadores-de-t...,Ofimática,"Opciones básicas de texto,Función Insertar"
6,Presentaciones básicas,En este curso aprenderás a crear presentacione...,https://edu.gcfglobal.org/es/presentaciones-ba...,https://www.youtube.com/watch?v=YMzDHZmCncQ&li...,https://edu.gcfglobal.org/es/presentaciones-ba...,Ofimática,Presentación básica
7,Programas de hojas de cálculo,Conoce cuáles son las herramientas básicas al ...,https://edu.gcfglobal.org/es/programas-de-hoja...,https://www.youtube.com/watch?v=ZBkkrnD7GBo&li...,https://edu.gcfglobal.org/es/programas-de-hoja...,Ofimática,"Composición de una hoja de cálculo,Fórmulas y ..."
8,Google Drive,Aprende en este curso gratuito de Google Drive...,https://edu.gcfglobal.org/es/google-drive/que-...,0,https://edu.gcfglobal.org/es/google-drive/,Ofimática,"Introducción a Google Drive, Gestión de archiv..."
9,Google Slides,En este curso te enseñaremos a usar este progr...,https://edu.gcfglobal.org/es/google-slides/goo...,0,https://edu.gcfglobal.org/es/google-slides/,Ofimática,"Introducción a Google Slides,Temas y plantilla..."


In [144]:
# Base de datos depurada
df.to_csv('drive/MyDrive/Laboratorios SW/Datos pagina/datosGCFDepurado.csv')